# Loads and Imports

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext Cython
%reload_ext Cython

In [2]:
#built-in
import pandas as pd
import numpy as np
import time
import datetime as dt
from pathlib import Path
import time
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
import pyarrow.parquet as pq
import seaborn as sns
import sys
import boto3

path = str(Path.home()) + '/rpi-canary-eval/temp'
git_path = str(Path.home()) + '/rpi-canary-eval'
sys.path.append(git_path + '/modules')


#custom
import data_process
from data_functions import make_date_to_MYR
from diabetes_onset import( process_canary,
get_tc, in_canary, diabet_onset, not_first_month_diabetes, 
                           match_data_per_month, 
                           filter_first_month_diabetes,
                           matched_population, 
                           propensity_func,
                           nearest_neighbor_func, 
                           random_matching_func, agglomerative )

from path import path_dict, col_seq_cc, col_seq_elx, col_seq_cc_all

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
np.random.seed(seed = 0)

/home/nneehal/.local/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Download and Import Latent Space data and corresponding Original Data from S3

In [3]:
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/ldata/m6_knn_no_cluster_only_latent/treated0_l.p ../../temp/ldata/treated0_l.p
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/ldata/m6_knn_no_cluster_only_latent/match0_l.p ../../temp/ldata/match0_l.p
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/odata/m6_knn_no_cluster_only_latent/treated0_o.p ../../temp/odata/treated0_o.p
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/odata/m6_knn_no_cluster_only_latent/match0_o.p ../../temp/odata/match0_o.p

In [4]:
import pickle
with open(path+'/ldata/match0_l.p', 'rb') as fp:
    controls_sick_latent = pickle.load(fp)
with open(path+'/ldata/treated0_l.p', 'rb') as fp:
    treated_sick_latent = pickle.load(fp)
with open(path+'/odata/match0_o.p', 'rb') as fp:
    controls_sick_original = pickle.load(fp)
with open(path+'/odata/treated0_o.p', 'rb') as fp:
    treated_sick_original = pickle.load(fp)

# Sort and Filter

In [5]:
controls = controls_sick_latent[controls_sick_latent.PERS_ID>"0"].sort_values(by=["PERS_ID", "MYR"])
treated = treated_sick_latent[treated_sick_latent.PERS_ID>"0"].sort_values(by=["PERS_ID","MYR"])

In [6]:
controls_orig = controls_sick_original[controls_sick_original.PERS_ID>"0"].sort_values(by=["PERS_ID", "MYR"])
treated_orig = treated_sick_original[treated_sick_original.PERS_ID>"0"].sort_values(by=["PERS_ID", "MYR"])

## Evaluation Quality Match

## Metric = ASMD

In [7]:
def ASMD_calculate(cols, tf, cf):
    ASMD = pd.DataFrame()
    for col in cols:
        t_mean = np.mean(tf[col].values)
        c_mean = np.mean(cf[col].values)
        t_var = np.var(tf[col].values)
        c_var = np.var(cf[col].values)
        d = np.abs((t_mean-c_mean)/np.sqrt((t_var+c_var)/2))
        ASMD.loc[0, col] = d
    return ASMD

In [8]:
def ASMD_final(cols, cdf, tdf):
    cf = cdf[cols]
    tf = tdf[cols]
    ASMD = ASMD_calculate(cols, tf, cf)
    return ASMD

### Latent Space

In [9]:
latent_cols = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16']

In [10]:
ASMD = ASMD_final(latent_cols, controls, treated)
ASMD

x1        x2        x3        x4       x5        x6        x7        x8        x9       x10       x11       x12       x13       x14       x15       x16
0  0.014097  0.016401  0.020501  0.020111  0.03243  0.014621  0.017504  0.003815  0.015463  0.021979  0.020203  0.012191  0.016408  0.013938  0.014488  0.007054

### Original Space with George's Features only

In [11]:
george_cols = ['AGE_AT_MIDMONTH', 'MBR_GNDR', 'ALLOW_AMT','CC_TOBACCO',  'CC_OBESITY',
                   'CC_HYPERTENSION', 'CC_HYPOTHYROID']

In [12]:
ASMD = ASMD_final(george_cols, controls_orig, treated_orig)
ASMD

AGE_AT_MIDMONTH  MBR_GNDR  ALLOW_AMT  CC_TOBACCO  CC_OBESITY  CC_HYPERTENSION  CC_HYPOTHYROID
0         0.047872  0.316595   0.063845    0.012503    0.285805         0.090724        0.055974

### Original Space with all the features

In [13]:
original_cols = controls_orig.columns.to_list()[2:]

In [14]:
ASMD = ASMD_final(original_cols, controls_orig, treated_orig)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


In [15]:
ASMD

AGE_AT_MIDMONTH  MBR_GNDR  MEDICAID_LOB  ALLOW_AMT  ALLOW_ER  ALLOW_IP   CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS    CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  CC_ASTHMA  CC_ATRIAL_FIB  CC_AUTISM  CC_BIPOLAR    CC_BPH  CC_BREAST_CANCER  CC_CATARACT  CC_CEREBRAL_PALSY  CC_CHRONIC_KIDNEY  CC_COLORECTAL_CANCER   CC_COPD  CC_CYSTIC_FIBROSIS  CC_DEPRESSION  CC_DEPRESSIVE_DISORDERS  CC_DEVELOP_DELAYS  CC_DIABETES  CC_DISAB_DX_CNT  CC_DX_CNT  CC_ENDOMETRIAL_CANCER  CC_EPILEPSY  CC_FIBROMYALGIA  CC_GLAUCOMA  CC_HEARING_IMPAIR  CC_HEART_FAILURE  CC_HEPATITIS_A  CC_HEPATITIS_B_ACUTE  CC_HEPATITIS_B_CHRONIC  CC_HEPATITIS_C_ACUTE  CC_HEPATITIS_C_CHRONIC  CC_HEPATITIS_C_UNSPECIFIED  CC_HEPATITIS_D  CC_HEPATITIS_E  CC_HEPATITIS_GEN  CC_HIP_FRACTURE  CC_HYPERLIPIDEMIA  CC_HYPERTENSION  CC_HYPOTHYROID  CC_INT_DISAB  CC_ISCHEMIC  CC_LEARN_DISAB  CC_LEUKEMIAS  CC_LIVER  CC_LUNG_CANCER  CC_MIGRAINE  CC_MOBILITY_IMPAIR  CC_MULTIPLE_SCLEROSIS  CC_MUSCULAR_DYSTROPHY  CC_OBESITY  \
0         0.047872  0.316595       0.26721   0.063845  0.015007  0.027246  0.092889      0.050043           0.097841  0.028029   0.019514             0.062658      0.058218    0.05864       0.011989   0.053834      0.0105  0.058554          0.117069      0.03233           0.010057           0.000054               0.01892  0.000617            0.060389       0.032233                 0.014575           0.013547     0.006951         0.002166   0.163055               0.014872     0.022918         0.017325     0.056798           0.036454          0.015309        0.022735              0.012309                0.009845              0.030165                0.061449                    0.031835             NaN             NaN          0.059715         0.021327           0.032796         0.090724        0.055974      0.026974     0.029041        0.003041      0.043163  0.023749        0.014916     0.110236            0.015641                0.04881                    NaN    0.285805   

   CC_OSTEOPOROSIS  CC_PERIPHERAL  CC_PERSONALITY  CC_PRESSURE  CC_PROSTATE_CANCER   CC_PTSD  CC_SCHIZOPHRENIA  CC_SCHIZOPHRENIA_OTHER  CC_SPINAL_CORD  CC_SPINA_BIFIDA  CC_STROKE  CC_TOBACCO  CC_TRAUMATIC_BRAIN  CC_VISUAL_IMPAIR  
0         0.005172       0.010327        0.025693     0.006984            0.014087  0.061748          0.027881                 0.05298        0.017158         0.020991   0.015783    0.012503            0.035797          0.002088